# BART Tutorial: Reconstruction of Dynamic Contrast Enhanced (DCE) MRI
## Practical Tools for Applying Compressed Sensing to 2D+Time Data

### Part 2

=========================================================================================================================

Hello, welcome back!

This is the **second** notebook of the Dynamic MRI tutorial. It is assumed that you are familiar with the materical in the first notebook, which is included in this repository and in named BART_DCE_demo.ipynb.

This notebook includes additional demonstrations of ``bart pics`` with these regularizers:
1. l1-wavelet in space
2. Total Variation (TV) in time
3. l1-wavelet in space + TVC in time

Notice: this notebook should be run using a Python3 kernel.

Efrat Shimron, UC Berkeley (efrat.s@berkeley.edu) and Jon Tamir, UT Austin (jtamir@utexas.edu).

December 2020


## BART's interfaces

Generally, we have two options for working with BART when using a Jupyter notebook with a python kernel:

1. Using the **Command Line Interface (CLI)**
2. Using **BART's python binder** - this enables reading the data from cfl into numpy arrays and working with numpy. 


### Introduction to BART's python binding

The `bart` module allows a user to execute any BART command-line tool via the following format:

`<outputs> = bart(<nargs>, <command>, <arguments>, ...)`

Where:

`<outputs>`: the output of a BART command

`<nargs>`: the number of outputs

`<command>`: the command string and necessary flags

`<arguments>`: the `<nargs>` data files passed in for use in the function

# Reconstruction 

We will use the `pics` tool to perform ESPIRiT-based parallel imaging and compressed sensing. The basic usage is

    bart pics [optimization options] [regularization options] kspace maps recon
    
See the full list with `bart pics -h`.
There are several built-in regularization terms and transforms. We can see the options by invoking the help:

In [ ]:
! bart pics -h

### Regularization

There are various options for regularization with bart pics. 
To see the different options for pics regularization, use `bart pics -Rh` flag:


In [ ]:
! bart pics -Rh

In the next few sections we will show how to reconstruct the 2D+Time data using different regularizers. 

## Regularization Example 1: *l*1-wavelet Regularizer in the Spatial Domain

#### option 1: using the CLI

In [1]:
! bart pics -d 5  -i 30 -p data/weights -R W:$(bart bitmask 0 1 2):0:0.005 data/ksp data/maps data/recon_l1wav

2 maps.
ESPIRiT reconstruction.
Size: 220320 Samples: 17236 Acc: 12.78
Linking args 1-2 of 4.
Linking args 1-2 of 4.
Linking args 1-2 of 4.
Linking args 1-2 of 4.
Calibration region...  (size: 1x5x2, pos: 0x31x89)
Scaling: 329041.093750! (max = 329041.093750/p90 = 329041.093750/median = 153375.046875)
Inverse scaling of the data: 329041.093750
l1-wavelet regularization: 0.005000
FISTA
Linking args 1-2 of 4.
Linking args 1-2 of 4.
Linking args 1-2 of 4.
Linking args 1-2 of 4.
Linking args 2-3 of 5.
Linking args 2-3 of 5.
#It 000: 1.000000   
#It 001: 0.266172   
#It 002: 0.116492   
#It 003: 0.066711   
#It 004: 0.042221   
#It 005: 0.027541   
#It 006: 0.019003   
#It 007: 0.014500   
#It 008: 0.011795   
#It 009: 0.009718   
#It 010: 0.008024   
#It 011: 0.006743   
#It 012: 0.005852   
#It 013: 0.005262   
#It 014: 0.004839   
#It 015: 0.004612   
#It 016: 0.004402   
#It 017: 0.004241   
#It 018: 0.004193   
#It 019: 0.004065   
#It 020: 0.003956   
#It 021: 0.003881   
#It 022: 0.0

In [ ]:
# load
recon_l1wav_CLI = cfl.readcfl('data/recon_l1wav')
recon_l1wav_CLI_sqz = recon_l1wav_CLI[0,:,:,0,0,0,0,0,0,0,:].squeeze()

# display - let's view the first 8 time frames (of 20)
fig = plt.figure(figsize = (10,8))
for t_ind in range(8):
    im_frame = recon_l1wav_CLI_sqz[:,:,t_ind].squeeze()
    plt.subplot(4,2,t_ind+1)
    plt.imshow(np.abs(im_frame),cmap="gray")
    plt.clim(0,5)
    plt.axis('off')
    plt.title('time frame {}'.format(t_ind+1))
plt.show()

In [ ]:
# Optional - view with Sigpy
recon_l1wav_CLI_transposed = np.moveaxis(recon_l1wav_CLI_sqz,-1,0) # to display with SigPy, move the temporal dim to the 0-dim.
recon_l1wav_CLI_rotated = np.rot90(recon_l1wav_CLI_transposed,2)

%matplotlib notebook
pl.ImagePlot(recon_l1wav_CLI_rotated)

#### option 2: using the python binder + cfl files 
You can use this option if you sapved the sens maps to memoery in the cfl format

In [ ]:
recon_l1wav = bart(1, 'pics -d 5 -i 30 -p {} -R W:7:0:0.005 {} {}'.format("data/weights","data/ksp","data/maps"))  # explanation: 7 = $(bart bitmask 0 1 2)


#### option3: using the python binder + numpy arrays
Here we use the numpy arrays that we created earlier: ksp and sens_maps. We can also read them from memory 

In [ ]:
# Remember that previously we ran these commands, which created the numpy arrays "ksp" and "sens_maps":
# ksp = cfl.readcfl('data/ksp')
# sens_maps = bart(1, 'ecalib -t 0.02 -c 0.5 -m 2', ksp_t_avg[:,:,:,:])

recon_l1wav = bart(1, 'pics -d 5 -i 30 -p {} -R W:7:0:0.005'.format("data/weights"), ksp , sens_maps)  # explanation: 7 = $(bart bitmask 0 1 2)



#### Display

In [ ]:
recon_l1wav_sqz = recon_l1wav[0,:,:,0,0,0,0,0,0,0,:].squeeze()

# display - let's view the first 8 time frames (of 20)
fig = plt.figure(figsize = (10,8))
for t_ind in range(8):
    im_frame = recon_l1wav_sqz[:,:,t_ind].squeeze()
    plt.subplot(4,2,t_ind+1)
    plt.imshow(np.abs(im_frame),cmap="gray")
    plt.clim(0,5)
    plt.axis('off')
    plt.title('time frame {}'.format(t_ind+1))
plt.show()

In [ ]:
# Optional - view with Sigpy

recon_l1wav_transposed = np.moveaxis(recon_l1wav_sqz,-1,0) # to display with SigPy, move the temporal dim to the 0-dim.
recon_l1wav_rotated = np.rot90(recon_l1wav_transposed,2)

%matplotlib notebook
pl.ImagePlot(recon_l1wav_rotated)

## Regularization Example 2: Total Variation (TV) in Time


In [ ]:
! bart pics -d 5 -i 100 -p data/weights -R T:$(bart bitmask 10):0:.04 data/ksp data/maps data/recon_tv


In [ ]:
# load
recon_tv = cfl.readcfl('data/recon_tv')
recon_tv_sqz = recon_tv[0,:,:,0,0,0,0,0,0,0,:].squeeze()

# display - let's view the first 8 time frames (of 20)
fig = plt.figure(figsize = (10,8))
for t_ind in range(8):
    im_frame = recon_tv_sqz[:,:,t_ind].squeeze()
    plt.subplot(4,2,t_ind+1)
    plt.imshow(np.abs(im_frame),cmap="gray")
    plt.clim(0,5)
    plt.axis('off')
    plt.title('time frame {}'.format(t_ind+1))
plt.show()

## Regularization Example 3: Two regularizers - *l*1-wavelet in Space and Total Variation (TV) in Time


#### option 1: using BART's CLI

In [ ]:
! bart pics -d 5 -i 100 -p data/weights -R W:$(bart bitmask 0 1 2):0:0.001 -R T:$(bart bitmask 10):0:.04 data/ksp data/maps data/recon_wavtv


In [ ]:
recon_wavTV = cfl.readcfl('data/recon_wavtv')

# view the recon
recon_wavTV_sqz = recon_wavTV[0,:,:,0,0,0,0,0,0,0,:].squeeze()
recon_wavTV_transpose = np.moveaxis(recon_wavTV_sqz,-1,0) # to display with SigPy, move the temporal dim to the 0-dim.
recon_wavTV_rotated = np.rot90(recon_wavTV_transpose,2)

pl.ImagePlot(recon_wavTV_rotated)

#### option 2: using bart's python binding

In [ ]:
sens_maps = cfl.readcfl('data/maps') 
print(sens_maps.dtype)
print(sens_maps.shape)

ksp = cfl.readcfl('data/ksp') 
print(ksp.dtype)
print(ksp.shape)

recon_wavTV = bart(1, 'pics -i 100 -p {} -R W:7:0:0.001 -R T:1024:0:.04 '.format("data/weights"),ksp , sens_maps)  


#### Display

In [ ]:
recon_wavTV_sqz = recon_wavTV[0,:,:,0,0,0,0,0,0,0,:].squeeze()

# display - let's view the first 8 time frames (of 20)
fig = plt.figure(figsize = (10,8))
for t_ind in range(8):
    im_frame = recon_wavTV_sqz[:,:,t_ind].squeeze()
    plt.subplot(4,2,t_ind+1)
    plt.imshow(np.abs(im_frame),cmap="gray")
    plt.clim(0,5)
    plt.axis('off')
    plt.title('time frame {}'.format(t_ind+1))

plt.suptitle('l1-Wavelet + TV regularization')
plt.show()

In [ ]:
# view the recon with Sigpy
recon_wavTV_sqz = recon_wavTV[0,:,:,0,0,0,0,0,0,0,:].squeeze()
recon_wavTV_transpose = np.moveaxis(recon_wavTV_sqz,-1,0) # to display with SigPy, move the temporal dim to the 0-dim.
recon_wavTV_rotated = np.rot90(recon_wavTV_transpose,2)

pl.ImagePlot(recon_wavTV_rotated)

## Compare the reconstructions of the different regularizers

In [ ]:
recon_l1wav = cfl.readcfl('data/recon_l1wav')
recon_tv = cfl.readcfl('data/recon_tv')
recon_wavTV = cfl.readcfl('data/recon_wavtv')
recon_LLR = cfl.readcfl('data/recon_LLR')  # Notice: this reconstruction was created in the first jupyter notebook


t_ind = 8  # choose a time frame

recon_l1wav_t = recon_l1wav[0,:,:,0,0,0,0,0,0,0,t_ind].squeeze()
recon_tv_t = recon_tv[0,:,:,0,0,0,0,0,0,0,t_ind].squeeze()
recon_wavTV_t = recon_wavTV[0,:,:,0,0,0,0,0,0,0,t_ind].squeeze()
recon_LLR_t = recon_LLR[0,:,:,0,0,0,0,0,0,0,t_ind].squeeze()

recs_array = np.concatenate((recon_l1wav_t,recon_l1wav_t,recon_wavTV_t,recon_LLR_t),axis=0)

fig = plt.figure(figsize=(8,4))
plt.subplot(2,2,1)
plt.imshow(np.abs(recon_l1wav_t),cmap="gray")
plt.clim(0,5)
plt.axis('off')
plt.title('l1-wav')

plt.subplot(2,2,2)
plt.imshow(np.abs(recon_tv_t),cmap="gray")
plt.clim(0,5)
plt.axis('off')
plt.title('TV')

plt.subplot(2,2,3)
plt.imshow(np.abs(recon_wavTV_t),cmap="gray")
plt.clim(0,5)
plt.axis('off')
plt.title('l1-wav + TV')

plt.subplot(2,2,4)
plt.imshow(np.abs(recon_LLR_t),cmap="gray")
plt.clim(0,5)
plt.axis('off')
plt.title('LLR')

plt.suptitle('time frame {}'.format(t_ind+1))
plt.show()


We hope you enjoyed this additional tutorial!

Efrat & Jon